from: https://github.com/SadmanSakib93/Stratified-k-fold-cross-validation-Image-classification-keras/blob/master/stratified_K_fold_CV.ipynb

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2023-04-19 04:21:18.901231: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 04:21:22.533572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
cwd = os.getcwd()
print("current working directory: ",cwd)

In [ ]:
"""from google.colab import drive
drive.mount('/content/drive')
!ls
import sys
# This is the path to where in google drive the code is stored!
root_path = '/content/drive/My Drive/'
sys.path.append(root_path)
"""

root_path = '/workspaces/proposal-ta/images/'

In [ ]:
### UBAH NAMA FILE MODEL SESUAI PENGUJIAN DI SINI
dataset_folder_name = os.path.join(root_path, '')
source_files = []
class_labels = ['a', 'ba', 'ca', 'da', 'ga', 
                'ja', 'ka','la','ma', 'na', 
                'nga', 'nya','pa', 'ra', 'sa', 
                'ta', 'wa', 'ya']
X = []
Y = []

img_rows, img_cols = 120, 120 # input image dimensions
train_path = os.path.join(dataset_folder_name, 'train')
validation_path = os.path.join(dataset_folder_name, 'validation')
test_path = os.path.join(dataset_folder_name, 'test')

In [5]:
# augmentasi data
# bisa jadi sebelum atau setelah split

In [6]:
def transfer_between_folders(source, dest, split_rate):
    """ Based on the split ratio this function moves some portion of the source folder to destination folder!

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest location

    """
    global source_files
    source_files = os.listdir(source)
    if(len(source_files) != 0):
        transfer_file_numbers = int(len(source_files)*split_rate)
        transfer_index = random.sample(
            range(0, len(source_files)), transfer_file_numbers)
        for each_index in transfer_index:
            shutil.move(os.path.join(source, str(source_files[each_index])), os.path.join(
                dest, str(source_files[each_index])))

    else:
        print("No file moved. Source empty!")


def transfer_all_class_between_folders(source, dest, split_rate):
    """ Transfer the files from source to dest for all the classes. This function calls the 'transfer_between_folders' to actually perform the transfer.

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest location

    """
    for label in class_labels:
        transfer_between_folders(os.path.join(dataset_folder_name, source, label),
                                 os.path.join(
                                     dataset_folder_name, dest, label),
                                 split_rate)


def my_metrics(y_true, y_pred):
    """ Calculate accuracy, precision, and f1 score of the model's prediction with respect to true labels.

        Args:
            y_true: list/array
                All true class labels
            y_pred: list/array
                All predicted class labels

        Returns:
            accuracy: float
                Accuracy measure of the model
            precision: float
                Precision measure of the model
            f1_Score: float
                F1-score measure of the model

    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    f1_Score = f1_score(y_true, y_pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1_Score))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1_Score

In [7]:
transfer_all_class_between_folders('test','train', 1.0)
transfer_all_class_between_folders('validation','train', 1.0)

In [8]:
transfer_all_class_between_folders('train', 'test', 0.20)

In [9]:
def prepare_name_with_labels(folder_name, dataset_type='train'):
    """ Prepare the file names (X) and the class labels (Y) from folder location of images.

        Args:
            folder_name: str
                Source folder's path

    """
    source_files = os.listdir(os.path.join(dataset_folder_name, dataset_type, folder_name))
    y_label = 0
    for i in range(len(class_labels)):
        if(folder_name == class_labels[i]):
            y_label = i
    for val in source_files:
        X.append(val)
        Y.append(y_label)

In [10]:
# Organize file names and class labels in X and Y variables
for i in range(len(class_labels)):
    prepare_name_with_labels(class_labels[i])

In [11]:
X = np.asarray(X)
Y = np.asarray(Y)

In [12]:
# arsitektur
batch_size = 32
epoch = 100
num_of_channels = 3
number_of_class_labels = len(class_labels)

In [13]:
"""
    Note that, this model structure is a very basic one. 
    To achieve better performance, you should change the model structure and hyperparameters according to your needs and data. 
    So, optimize the structure of the model!
"""

def get_model():
    activation_function = 'relu'

    model = Sequential()
    model.add(Conv2D(64, (3,3), padding='same',
                     activation=activation_function, input_shape=(img_rows, img_cols, num_of_channels)))
    model.add(Conv2D(64, (3,3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    model.add(Conv2D(32, (3,3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(32, (3,3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    model.add(Conv2D(16, (3,3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(16, (3,3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    #model.add(Conv2D(64, (3,3), padding='same',
    #                 activation=activation_function))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
    
    #model.add(Conv2D(64, (3,3), padding='same',
    #                 activation=activation_function))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    #model.add(Conv2D(128, (3,3), padding='same',
    #                 activation=activation_function))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation=activation_function))
    #model.add(Dropout(0.1))
    #model.add(Dense(32, activation=activation_function))
    #model.add(Dropout(0.1))
    #model.add(Dense(16, activation=activation_function))
    #model.add(Dropout(0.1))
    model.add(Dense(number_of_class_labels, activation='softmax'))

    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


model = get_model()
model.summary()


In [14]:
"""model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(120, 120, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    #tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(18, activation='softmax')
])

optimizer=tf.keras.optimizers.Adam(learning_rate=0.005)

model.compile(
    #optimizer='adam', 
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

model.summary()"""

"model = tf.keras.Sequential([\n    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(120, 120, 3)),\n    tf.keras.layers.MaxPooling2D(2, 2),\n    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),\n    tf.keras.layers.MaxPooling2D(2, 2),\n    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),\n    tf.keras.layers.MaxPooling2D(2, 2),\n    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),\n    tf.keras.layers.MaxPooling2D(2, 2),\n    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),\n    tf.keras.layers.MaxPooling2D(2, 2),\n    #tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),\n    #tf.keras.layers.MaxPooling2D(2, 2),\n    tf.keras.layers.Flatten(),\n    #tf.keras.layers.Dropout(0.2),\n    tf.keras.layers.Dense(128, activation='relu'),\n    tf.keras.layers.Dense(18, activation='softmax')\n])\n\noptimizer=tf.keras.optimizers.Adam(learning_rate=0.005)\n\nmodel.compile(\n    #optimizer='adam', \n    optimizer=optimizer,\n    loss='categorical_cross

In [15]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, Y)
fold_num = 0
for train_index, val_index in skf.split(X, Y):
    # First cut all images from validation to train (if any exists)
    transfer_all_class_between_folders('validation', 'train'
                                       , 1.0)
    fold_num += 1
    print("Results for fold", fold_num)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for each_index in range(len(X_val)):
        class_label = ''
        for i in range(len(class_labels)):
            if(Y_val[each_index] == i):
                class_label = class_labels[i]
        # Then, copy the validation images to the validation folder
        shutil.move(os.path.join(dataset_folder_name, 'train', class_label, X_val[each_index]),
                    os.path.join(dataset_folder_name, 'validation', class_label, X_val[each_index]))

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.20,
        fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)
    
    # fit model
    history = model.fit(train_generator,
                        epochs=epoch)

    predictions = model.predict(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    
    # evaluate validation performance
    print("***Performance on Validation data***")
    val_acc, val_prec, val_fScore = my_metrics(true_classes, y_predictions)

Results for fold 1
Found 1070 images belonging to 18 classes.
Found 207 images belonging to 18 classes.
Epoch 1/100


2023-04-19 04:21:33.135058: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-19 04:21:35.167155: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117964800 exceeds 10% of free system memory.
2023-04-19 04:21:35.642270: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 114065408 exceeds 10% of free system memory.
2023-04-19 04:21:36.369579: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 114065408 exceeds 10% of free system memory.
2023-04-19 04:21:36.474108: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117964800 exceeds 10% of free system memory.


 1/34 [..............................] - ETA: 2:14 - loss: 2.8970 - accuracy: 0.0000e+00

2023-04-19 04:21:37.302046: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117964800 exceeds 10% of free system memory.


34/34 [==============================] - 53s 1s/step - loss: 2.8897 - accuracy: 0.0710
Epoch 2/100
34/34 [==============================] - 50s 1s/step - loss: 2.8577 - accuracy: 0.1673
Epoch 3/100
34/34 [==============================] - 49s 1s/step - loss: 2.4576 - accuracy: 0.3579
Epoch 4/100
34/34 [==============================] - 50s 1s/step - loss: 1.2398 - accuracy: 0.6355
Epoch 5/100
34/34 [==============================] - 50s 1s/step - loss: 0.8215 - accuracy: 0.7561
Epoch 6/100
34/34 [==============================] - 49s 1s/step - loss: 0.6645 - accuracy: 0.8065
Epoch 7/100
34/34 [==============================] - 50s 1s/step - loss: 0.6023 - accuracy: 0.8159
Epoch 8/100
34/34 [==============================] - 49s 1s/step - loss: 0.5172 - accuracy: 0.8430
Epoch 9/100
34/34 [==============================] - 50s 1s/step - loss: 0.4620 - accuracy: 0.8636
Epoch 10/100
34/34 [==============================] - 50s 1s/step - loss: 0.4162 - accuracy: 0.8804
Epoch 11/100
34/34 [=

...

7/7 [==============================] - 2s 249ms/step
***Performance on Validation data***
Accuracy  : 1.0
Precision : 1.0
f1Score : 1.0
[[11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 12  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 12  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 11  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 12  0  0  0]
 [ 0  0  

2023-04-19 09:58:06.586105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


34/34 [==============================] - 51s 1s/step - loss: 4.4545e-04 - accuracy: 1.0000
Epoch 2/100
34/34 [==============================] - 51s 1s/step - loss: 1.2483e-04 - accuracy: 1.0000
Epoch 3/100
34/34 [==============================] - 51s 2s/step - loss: 2.4988e-04 - accuracy: 1.0000
Epoch 4/100
34/34 [==============================] - 52s 2s/step - loss: 2.2863e-04 - accuracy: 1.0000
Epoch 5/100
34/34 [==============================] - 52s 2s/step - loss: 6.9180e-05 - accuracy: 1.0000
Epoch 6/100
34/34 [==============================] - 51s 1s/step - loss: 2.8262e-05 - accuracy: 1.0000
Epoch 7/100
34/34 [==============================] - 52s 2s/step - loss: 1.0220e-04 - accuracy: 1.0000
Epoch 8/100
34/34 [==============================] - 51s 2s/step - loss: 7.8013e-05 - accuracy: 1.0000
Epoch 9/100
34/34 [==============================] - 52s 2s/step - loss: 8.4067e-05 - accuracy: 1.0000
Epoch 10/100
34/34 [==============================] - 51s 2s/step - loss: 5.5464e-04 

2023-04-19 11:25:24.749042: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - 2s 269ms/step
***Performance on Validation data***
Accuracy  : 1.0
Precision : 1.0
f1Score : 1.0
[[11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 12  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 12  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 11  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 12  0  0  0]
 [ 0  0  

In [16]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False 
)
predictions = model.predict(test_generator, verbose=1) 
y_predictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

test_acc, test_prec, test_fScore = my_metrics(true_classes, y_predictions)

==============TEST RESULTS============
Found 252 images belonging to 18 classes.


2023-04-19 11:25:27.159912: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


8/8 [==============================] - 3s 336ms/step
Accuracy  : 0.9285714285714286
Precision : 0.9339870777370777
f1Score : 0.9290312454680271
[[13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 0 12  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  1 12  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0]
 [ 0  0  0 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 13  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0 13  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 14  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 14  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0 13  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  1  0  0  0  0  0 11  0  0  1  0  0  0  0]
 [ 0  1  0  0  1  0  0  0  0  0  0 12  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 13  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 14  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 13  0  0  1]
 

In [17]:
from sklearn.metrics import classification_report
clr = classification_report(true_classes, y_predictions,
                            labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 
                            target_names=['a', 'ba','ca','da','ga','ja','ka','la','ma','na','nga','nya','pa','ra','sa','ta','wa','ya'])
print(clr)

              precision    recall  f1-score   support

           a       1.00      0.93      0.96        14
          ba       0.75      0.86      0.80        14
          ca       1.00      0.86      0.92        14
          da       0.88      1.00      0.93        14
          ga       0.81      0.93      0.87        14
          ja       1.00      0.93      0.96        14
          ka       1.00      1.00      1.00        14
          la       0.93      0.93      0.93        14
          ma       1.00      1.00      1.00        14
          na       1.00      0.93      0.96        14
         nga       1.00      0.79      0.88        14
         nya       0.92      0.86      0.89        14
          pa       0.93      0.93      0.93        14
          ra       0.93      1.00      0.97        14
          sa       0.93      0.93      0.93        14
          ta       0.93      0.93      0.93        14
          wa       0.93      0.93      0.93        14
          ya       0.88    

In [18]:
MODEL_FILENAME = root_path+"/modelpengujian17_3x3_arsitektur2.h5"
model.save(MODEL_FILENAME)